<a href="https://colab.research.google.com/github/farshid101/Thesis-2024/blob/main/h2o_danube_classifier_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pip

In [1]:
!pip install sentencepiece -q
!pip install transformers -q
!pip install accelerate -q
!pip install peft -q
!pip install bitsandbytes -q
!pip install lightning -q
!pip install flash-attn --no-build-isolation -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 45.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.2/869.2 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 82.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


# Imports

In [2]:
import numpy as np
import pandas as pd

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    get_linear_schedule_with_warmup,
    get_cosine_schedule_with_warmup
)

import os
import time
import zipfile
import urllib.request
from pathlib import Path
from tqdm.auto import tqdm

import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score
import lightning as L

from peft import (
    get_peft_config,
    get_peft_model,
    LoraConfig,
    TaskType
)

tqdm.pandas()
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Get data

In [ ]:
# url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
# zip_path = "sms_spam_collection.zip"
# extracted_path = "sms_spam_collection"
# data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

# def download_and_unzip(url, zip_path, extracted_path, data_file_path):
#     if data_file_path.exists():
#         print(f"{data_file_path} already exists. Skipping download and extraction.")
#         return

#     # Downloading the file
#     with urllib.request.urlopen(url) as response:
#         with open(zip_path, "wb") as out_file:
#             out_file.write(response.read())

#     # Unzipping the file
#     with zipfile.ZipFile(zip_path, "r") as zip_ref:
#         zip_ref.extractall(extracted_path)

#     # Add .tsv file extension
#     original_file_path = Path(extracted_path) / "SMSSpamCollection"
#     os.rename(original_file_path, data_file_path)
#     print(f"File downloaded and saved as {data_file_path}")

# download_and_unzip(url, zip_path, extracted_path, data_file_path)

sms_spam_collection/SMSSpamCollection.tsv already exists. Skipping download and extraction.


In [3]:
df = pd.read_csv("/content/Metro_testing_data.csv")
df

,comment,Compound to sentiment
0,ধন্যবাদ ভাই। এয়ারপোর্টে যেতে হলে উত্তরা কোন...,positive
1,আশা করি সেপ্টেম্বরের প্রথম সপ্তাহে দেখতে পাব...,positive
2,কোনটাই সহজ হবে না। সবটাই এয়ারপোর্ট থেকে দূর...,negative
3,Agargaon to Uttara,neutral
4,nice to see you in Bangladesh again. You reall...,positive
...,...,...
2002,এটি একটি উপন্যাস যে আপনি ভূগর্ভস্থ যান না যদিও...,positive
2003,জাপান থেকে আমাকে মাফ করবেন! ঢাকা মেট্রো চালুর ...,positive
2004,আমি অত্যন্ত আনন্দিত যে জাপানি রেলওয়ে প্রযুক্ত...,positive
2005,সাম্প্রতিক বলে মনে হচ্ছে। আশা করি বাকিটাও তাড়...,positive


In [4]:
import pandas as pd

# Function to rename a single column
def change_column_name(df, old_name, new_name):
    df = df.rename(columns={old_name: new_name})
    return df

# Assuming df is your DataFrame
df = change_column_name(df, old_name="Compound to sentiment", new_name="target")
df = change_column_name(df, old_name="comment", new_name="text")
print(df.head())  # Display the first few rows of the DataFrame
#text

                                                text    target
0    ধন্যবাদ ভাই।  এয়ারপোর্টে যেতে হলে উত্তরা কোন...  positive
1    আশা করি সেপ্টেম্বরের প্রথম সপ্তাহে দেখতে পাব...  positive
2    কোনটাই সহজ হবে না।  সবটাই এয়ারপোর্ট থেকে দূর...  negative
3                                 Agargaon to Uttara   neutral
4  nice to see you in Bangladesh again. You reall...  positive


In [5]:
df.target.value_counts()

,count
target,
positive,1043
neutral,770
negative,194


In [8]:
# Assuming df is your DataFrame

# Drop rows where the 'target' column is 'negative'
df_filtered = df[df['target'] != 'negative']

# Display the result
df=df_filtered


In [9]:
df.target.value_counts()

,count
target,
positive,1043
neutral,770


In [14]:
import pandas as pd

def create_balanced_dataset(df):
    # Count the instances of "positive" and "neutral"
    num_positive = df[df["target"] == "positive"].shape[0]
    num_neutral = df[df["target"] == "neutral"].shape[0]

    # Determine the number to sample (use the minimum count)
    num_to_sample = min(num_positive, num_neutral)

    # Randomly sample "neutral" instances
    neutral_subset = df[df["target"] == "neutral"].sample(num_to_sample, random_state=123)

    # Randomly sample "positive" instances (in case there are more than needed)
    positive_subset = df[df["target"] == "positive"].sample(num_to_sample, random_state=123)

    # Combine the "positive" and "neutral" subsets
    balanced_df = pd.concat([neutral_subset, positive_subset])

    return balanced_df

# Example usage
balanced_df = create_balanced_dataset(df)
print(balanced_df["target"].value_counts())


target
neutral     770
positive    770
Name: count, dtype: int64


In [15]:
balanced_df['target'] = df.target.map({'positive': 1, 'neutral': 0})

In [16]:
def random_split(df, train_frac, validation_frac):
    # Shuffle the entire DataFrame
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # Calculate split indices
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    # Split the DataFrame
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(balanced_df, 0.7, 0.1)
# Test size is implied to be 0.2 as the remainder

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

# Load tokenizer

In [32]:
# Initialize tokenizer with model ID and authentication token
# model_id = 'h2oai/h2o-danube-1.8b-chat'
model_id ='openai-community/openai-gpt'
hf_token = 'hf_' # Replace your token here on huggingface

tokenizer = AutoTokenizer.from_pretrained(model_id)

# Set padding token to end-of-sequence token and configure padding side
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Split data

In [18]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
val = pd.read_csv('validation.csv')

In [19]:
train['text'] = tokenizer.bos_token + train['text']
test['text'] = tokenizer.bos_token + test['text']
val['text'] = tokenizer.bos_token + val['text']

In [20]:
sample = tokenizer(train.text[0], add_special_tokens=False).input_ids
tokenizer.decode(sample)

'<s> করব আমি খেলা।'

# Dataset and DataLoader

In [21]:
class CustomDataset(Dataset):
    def __init__(self, texts, targets):
        self.texts = texts
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __getitem__(self, idx):
        text = self.texts[idx]
        target = self.targets[idx]
        return text, target

    def __len__(self):
        return len(self.targets)

In [22]:
# Set seed for reproducibility
L.seed_everything(seed=252)

# Create train dataset and dataloader
train_dataset = CustomDataset(
    texts=train['text'].values.tolist(),
    targets=train['target'].values.tolist()
)
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=2,
    shuffle=True,
    drop_last=True
)

# Create test dataset and dataloader
test_dataset = CustomDataset(
    texts=test['text'].values.tolist(),
    targets=test['target'].values.tolist()
)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=16,
    shuffle=False,
    drop_last=False
)

# Create validation dataset and dataloader
val_dataset = CustomDataset(
    texts=val['text'].values.tolist(),
    targets=val['target'].values.tolist()
)
val_dataloader = DataLoader(
    dataset=val_dataset,
    batch_size=16,
    shuffle=False,
    drop_last=False
)

INFO: Seed set to 252
INFO:lightning.fabric.utilities.seed:Seed set to 252


# Tokenization function

In [23]:
def tokenize_text(text):
    """
    Tokenize the text and return PyTorch tensors with dynamic padding
    """
    encodings = tokenizer(
        text,
        return_tensors='pt',
        padding='longest',  # Dynamically pad each batch to the length of the longest sequence
        add_special_tokens=False
    )

    return encodings


# Architecture

In [24]:
def disable_dropout(model: torch.nn.Module):
    """Disable dropout in a model."""
    for module in model.modules():
        if isinstance(module, torch.nn.Dropout):
            module.p = 0

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        # Get LLM configuration
        config = AutoConfig.from_pretrained(model_id)

        # LoRA config
        peft_config = LoraConfig(
            task_type=TaskType.CAUSAL_LM,
            inference_mode=False,
            r=8,
            lora_alpha=16,
            target_modules='all-linear',
            lora_dropout=0.
        )

        # Load pre-trained language model with specific configurations
        self.backbone = AutoModelForCausalLM.from_pretrained(
            model_id,
            device_map="cuda",
            low_cpu_mem_usage=True,
            trust_remote_code=True,
        )

        # Replace language model head with an identity function
        self.backbone.lm_head = nn.Identity()

        # Apply LoRA
        self.backbone = get_peft_model(self.backbone, peft_config)
        self.backbone.print_trainable_parameters()

        # Define classification head
        self.cls_head = nn.Sequential(
            nn.Linear(config.hidden_size, 768),
            nn.ReLU(),
            nn.LayerNorm(768),
            nn.Linear(768, 2)
        )

    def forward(self, input_ids, attention_mask):
        x = self.backbone(input_ids, attention_mask).logits  # get last hidden state
        logits = self.cls_head(x)[:, -1, :]  # Apply classification head to the last token's output
        return logits


# Optimizer and Scheduler

In [25]:
def get_optimizer(model, learning_rate=0.0001, diff_lr=0.00001, weight_decay=0.01):
    """
    Get optimizer with different learning rates for specified layers.

    Args:
        model (torch.nn.Module): The neural network model.
        learning_rate (float): Learning rate for non-differential layers.
        diff_lr (float): Learning rate for differential layers.
        weight_decay (float): Weight decay (decoupled from L2 penalty) for optimizer.

    Returns:
        torch.optim.AdamW: Optimizer for the model.
    """

    # Define parameters with different learning rates and weight decay
    no_decay = ['bias', 'LayerNorm.weight']
    differential_layers = ['backbone']

    optimizer = torch.optim.AdamW(
            [
                {
                    "params": [
                        param
                        for name, param in model.named_parameters()
                        if (not any(layer in name for layer in differential_layers))
                        and (not any(nd in name for nd in no_decay))
                    ],
                    "lr": learning_rate,
                    "weight_decay": weight_decay,
                },
                {
                    "params": [
                        param
                        for name, param in model.named_parameters()
                        if (not any(layer in name for layer in differential_layers))
                        and (any(nd in name for nd in no_decay))
                    ],
                    "lr": learning_rate,
                    "weight_decay": 0,
                },
                {
                    "params": [
                        param
                        for name, param in model.named_parameters()
                        if (any(layer in name for layer in differential_layers))
                        and (not any(nd in name for nd in no_decay))
                    ],
                    "lr": diff_lr,
                    "weight_decay": weight_decay,
                },
                {
                    "params": [
                        param
                        for name, param in model.named_parameters()
                        if (any(layer in name for layer in differential_layers))
                        and (any(nd in name for nd in no_decay))
                    ],
                    "lr": diff_lr,
                    "weight_decay": 0,
                },
            ],
            lr=learning_rate,
            weight_decay=weight_decay,
    )

    return optimizer

# Hyperameters

In [26]:
num_epochs = 2
learning_rate = 0.0002
diff_lr = 0.00001
warmup_steps = 0
seed = 252
weight_decay = 0.01
accumulation_steps = 2

# Fine-tuning

In [27]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


In [28]:
# Set seed for reproducibility
L.seed_everything(seed=seed)

# Instantiate the neural network model
model = Net()
model.to(device)  # Move model to the device

# Display the names of trainable parameters
print('Here are the trainable parameters:')
for n, p in model.named_parameters():
    if p.requires_grad:
        print(n)

# Get the optimizer
optimizer = get_optimizer(
    model,
    learning_rate=learning_rate,
    diff_lr=diff_lr,
    weight_decay=weight_decay
)

# Set up the scheduler for learning rate adjustment
scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=warmup_steps,
    num_training_steps=num_epochs*len(train_dataloader)
)

INFO: Seed set to 252
INFO:lightning.fabric.utilities.seed:Seed set to 252


config.json:   0%|          | 0.00/710 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.66G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

trainable params: 8,650,752 || all params: 1,757,932,032 || trainable%: 0.4921
Here are the trainable parameters:
backbone.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight
backbone.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight
backbone.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight
backbone.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight
backbone.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight
backbone.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight
backbone.base_model.model.model.layers.0.self_attn.o_proj.lora_A.default.weight
backbone.base_model.model.model.layers.0.self_attn.o_proj.lora_B.default.weight
backbone.base_model.model.model.layers.0.mlp.gate_proj.lora_A.default.weight
backbone.base_model.model.model.layers.0.mlp.gate_proj.lora_B.default.weight
backbone.base_model.model.model.layers.0.mlp.up_proj.lora_A.default.weight
backbone.base_mod

In [29]:
scaler = GradScaler()

start_time = time.time()
for epoch in range(num_epochs):

    for batch_idx, batch in enumerate(train_dataloader):

        model.train()

        prompt, targets = batch

        encodings = tokenize_text(prompt)

        input_ids = encodings['input_ids'].to(device)
        attention_mask = encodings['attention_mask'].to(device)
        targets = targets.to(device)

        # Perform forward pass with autocast for mixed precision training
        with autocast():
            logits = model(input_ids, attention_mask)
            loss = F.cross_entropy(logits, targets) / accumulation_steps

        scaler.scale(loss).backward()

        # Backward pass, optimization step, and learning rate adjustment
        if not batch_idx % accumulation_steps:
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            optimizer.zero_grad()

        # Logging training progress after 100 steps
        if not (batch_idx + 1) % 100:
            print(
                f'Epoch: {epoch+1} / {num_epochs}'
                f'| Batch: {batch_idx+1}/{len(train_dataloader)}'
                f'| Loss: {loss.item():.4f}'
            )

end_time = time.time()
training_time = (end_time - start_time) / 60
print(f'Total training time: {training_time:.2f} min')

<ipython-input-29-1912edc03be1>:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-29-1912edc03be1>:19: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch: 1 / 2| Batch: 100/539| Loss: 0.0551
Epoch: 1 / 2| Batch: 200/539| Loss: 0.5890
Epoch: 1 / 2| Batch: 300/539| Loss: 0.2338
Epoch: 1 / 2| Batch: 400/539| Loss: 0.2935
Epoch: 1 / 2| Batch: 500/539| Loss: 0.2610
Epoch: 2 / 2| Batch: 100/539| Loss: 0.1414
Epoch: 2 / 2| Batch: 200/539| Loss: 0.2006
Epoch: 2 / 2| Batch: 300/539| Loss: 0.0394
Epoch: 2 / 2| Batch: 400/539| Loss: 0.0855
Epoch: 2 / 2| Batch: 500/539| Loss: 0.2970
Total training time: 4.44 min


# Evaluation

In [30]:
def calc_accuracy(dataloader, type):
    with torch.no_grad():
        model.eval()
        pred_scores = []
        actual_scores = []
        for batch in tqdm(dataloader, total=len(dataloader), desc=f'Calc {type} accuracy'):
            prompt, targets = batch
            encodings = tokenize_text(prompt)

            input_ids = encodings['input_ids'].to(device)
            attention_mask = encodings['attention_mask'].to(device)

            with autocast():
                logits = model(input_ids, attention_mask)
                pred_score = F.softmax(logits, dim=-1).argmax(dim=-1).cpu().detach().numpy().tolist()
                pred_scores.extend(pred_score)
                actual_scores.extend(targets.numpy().tolist())

        pred_scores = np.array(pred_scores)
        accuracy = accuracy_score(actual_scores, pred_scores)

        return accuracy

In [31]:
train_acc = calc_accuracy(train_dataloader, type='train')
test_acc = calc_accuracy(test_dataloader, type='test')
val_acc = calc_accuracy(val_dataloader, type='val')

print('Train accuracy:', train_acc)
print('Test accuracy:', test_acc)
print('Val accuracy:', val_acc)

Calc train accuracy:   0%|          | 0/539 [00:00<?, ?it/s]

<ipython-input-30-9fd11120be9e>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


Calc test accuracy:   0%|          | 0/20 [00:00<?, ?it/s]

<ipython-input-30-9fd11120be9e>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Calc val accuracy:   0%|          | 0/10 [00:00<?, ?it/s]

<ipython-input-30-9fd11120be9e>:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Train accuracy: 0.8042671614100185
Test accuracy: 0.6590909090909091
Val accuracy: 0.6623376623376623


| Model        | Weights   | Trainable token | Trainable layers        | Context length                            | CPU/GPU | Training time | Training acc | Validation acc | Test acc |
|--------------|-----------|-----------------|-------------------------|-------------------------------------------|---------|---------------|--------------|----------------|-----------|
| h20-danube (1.8 B)| instruct   | last            | LoRA                    | dynamic padding (batch-wise)        | T4 (Colab free)    | 2.76 min      | 99.71%       | 99.32%         | 97.33%    |